<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myStocks_Portfolio_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Import the function from the script
import myStocks_Portfolio_Analysis_v2

# Call the function to get the DataFrame
df_portfolio_features = myStocks_Portfolio_Analysis_v2.get_portfolio_features_v2()
cols = ['Symbol', 'AvgCost', 'Close', 'FTT', 'Momentm', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR%', 'RRR Ind', 'CurrAlloc%', 'Criteria', 'Strategy', 'Category']
df_portfolio_features = df_portfolio_features[cols]

Run date time (IST): 2024-08-07 10:21:48
-------------------
qualified stocks: 79
with latest results: 77
still star stocks: 45
-------------------
Initial Investment:  88.90 L
CY Investment:  1.02 C
Reserve:  1.83 K
Current:  1.12 C
-------------------
Today PnL: 38.45 K (0.35%)
Current PnL: 4.38 L (4.29%)
CY Booked + Current PnL: 9.52 L (9.33%)
-------------------
Total profit:  9.39 L
Total loss:  -5.01 L
-------------------
Total Booked + Current PnL: 22.63 L (25.45%)
Total Booked PnL: 18.25 L (20.53%)
Curr Year Booked PnL: 5.14 L (4.61%)
Prev Year Booked PnL: 13.11 L (14.75%)
Est FTT:  1.66 C
Est FTT PnL: 54.37 L (48.75%)
-------------------
Est LTT:  2.11 C
Est LTT PnL: 99.65 L (89.35%)
Deployed:  89.02 L
Current:  1.12 C
CAGR/XIRR %: 25.31%


In [4]:
# all portfolio
df_portfolio_features = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
data_table.DataTable(df_portfolio_features, include_index=False)

,Symbol,AvgCost,Close,FTT,Momentm,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR%,RRR Ind,CurrAlloc%,Criteria,Strategy,Category
21,DABUR,523.35,634.45,710.00,UP,111029.0,19443.0,13224.0,0.84,21.23,11.91,35.66,26.0,1.47,1.00,AV40,BTT,FMCG
58,ROSSARI,770.50,889.10,965.00,UP,168929.0,22534.0,14427.0,1.50,15.39,8.54,25.24,28.0,1.56,1.51,AVSP,NTT,CHEMICALS
29,HCLTECH,1349.00,1601.60,1821.00,UP,120120.0,18945.0,16456.0,1.19,18.72,13.70,34.99,151.0,1.15,1.08,AV40,BTT,IT
46,LALPATHLAB,2519.90,3180.00,3689.00,UP,114480.0,23764.0,18328.0,0.37,26.20,16.01,46.39,50.0,1.30,1.03,AV40N,NTT,HEALTHCARE
50,METROPOLIS,1399.27,2040.00,2345.00,UP,136680.0,42929.0,20434.0,-0.67,45.79,14.95,67.59,52.0,2.10,1.23,AVR,NTT,HEALTHCARE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,CAMPUS,297.39,294.90,631.85,UP,136539.0,-1153.0,156009.0,0.56,-0.84,114.26,112.47,170.0,-0.01,1.22,AVSP,ATH,FOOTWEAR
76,VAIBHAVGBL,350.21,308.95,670.00,NaN,134084.0,-17907.0,156691.0,0.73,-11.78,116.86,91.31,-13.0,-0.11,1.20,AVR,NTT,JEWELLERY
10,BANDHANBNK,227.40,203.42,400.00,NaN,168228.0,-19832.0,162576.0,1.16,-10.55,96.64,75.90,-7.0,-0.12,1.51,AVSP,NTT,BANKS
68,TANLA,969.42,906.90,2007.31,DOWN,150545.0,-10379.0,182671.0,1.76,-6.45,121.34,107.06,-2.0,-0.06,1.35,AAR,ATH,IT


In [5]:
# AAR analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AAR') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,AvgCost,Close,FTT,Momentm,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR%,RRR Ind,CurrAlloc%,Criteria,Strategy,Category
43,JUBLFOOD,487.35,605.00,703.00,UP,135520.0,26354.0,21954.0,1.21,24.14,16.20,44.25,22.0,1.20,1.22,AAR,NTT,RESTAURANTS
51,NEOGEN,1489.90,1565.50,1901.29,NaN,106454.0,5141.0,22834.0,0.95,5.07,21.45,27.61,16.0,0.23,0.95,AAR,ATH,CHEMICALS
64,SONACOMS,578.05,651.40,800.00,UP,113995.0,12836.0,26002.0,0.01,12.69,22.81,38.40,33.0,0.49,1.02,AAR,NTT,AUTO
22,DEN,52.79,51.42,75.00,UP,82272.0,-2192.0,37730.0,1.90,-2.60,45.86,42.07,5.0,-0.06,0.74,AAR,NTT,ENTERTAINMENT
27,HAPPSTMNDS,919.80,776.95,1496.99,NaN,81580.0,-14999.0,75608.0,-0.06,-15.53,92.68,62.75,-21.0,-0.20,0.73,AAR,ATH,IT
68,TANLA,969.42,906.90,2007.31,DOWN,150545.0,-10379.0,182671.0,1.76,-6.45,121.34,107.06,-2.0,-0.06,1.35,AAR,ATH,IT


In [6]:
# AVSP analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AVSP') ]
data_table.DataTable(tmp_df, include_index=False)

,Symbol,AvgCost,Close,FTT,Momentm,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR%,RRR Ind,CurrAlloc%,Criteria,Strategy,Category
58,ROSSARI,770.50,889.10,965.00,UP,168929.0,22534.0,14427.0,1.50,15.39,8.54,25.24,28.0,1.56,1.51,AVSP,NTT,CHEMICALS
40,JIOFIN,319.49,313.90,387.00,NaN,125874.0,-2241.0,29316.0,0.82,-1.75,23.29,21.13,139.0,-0.08,1.13,AVSP,BTT,FINANCE
16,BIOCON,291.09,338.00,397.00,UP,174408.0,24206.0,30452.0,1.05,16.12,17.46,36.38,56.0,0.79,1.56,AVSP,NTT,PHARMA
62,SHALBY,271.92,267.40,327.00,UP,147872.0,-2500.0,32961.0,-0.22,-1.66,22.29,20.26,49.0,-0.08,1.33,AVSP,NTT,HEALTHCARE
75,UTIAMC,946.90,988.45,1207.00,UP,156175.0,6565.0,34530.0,1.14,4.39,22.11,27.47,41.0,0.19,1.40,AVSP,NTT,FINANCE
17,BOROLTD,348.25,361.95,462.00,NaN,145504.0,5508.0,40217.0,0.10,3.93,27.64,32.66,118.0,0.14,1.30,AVSP,NTT,GLASS
15,BERGEPAINT,558.54,536.30,680.00,DOWN,170007.0,-7050.0,45545.0,-0.81,-3.98,26.79,21.75,-5.0,-0.15,1.52,AVSP,NTT,PAINTS
7,AWL,370.20,370.75,485.00,DOWN,150895.0,224.0,46506.0,1.74,0.15,30.82,31.01,7.0,0.00,1.35,AVSP,NTT,FMCG
65,STARHEALTH,596.56,571.45,761.00,UP,153149.0,-6729.0,50800.0,0.08,-4.21,33.17,27.56,-1.0,-0.13,1.37,AVSP,NTT,INSURANCE
39,IRCTC,904.87,924.25,1269.00,UP,144183.0,3023.0,53780.0,0.55,2.14,37.30,40.24,41.0,0.06,1.29,AVSP,BTT,TRAVEL
